# Final Modeling

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

## Dataset craetion

In [2]:
df = pd.read_feather('removed_duplicated_columns.feather')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 726288 entries, 0 to 726287
Columns: 767 entries, HKTLMYY to TLJYWBE
dtypes: float64(742), int64(21), object(4)
memory usage: 4.2+ GB


In [3]:
target = 'TLJYWBE'
df[target] = df[target].apply(lambda v: v >= 1e-5)
df[target].value_counts()

False    726224
True         64
Name: TLJYWBE, dtype: int64

In [8]:
features = [c for c in df.columns if c != target]
training_features, test_features, train_target, test_target = train_test_split(df[features], df[target], test_size=0.3, stratify=df[target])

In [9]:
print(f'Training set shape: {training_features.shape}')
print(f'Training set target shape: {train_target.shape}')
print(f'Test set shape: {test_features.shape}')
print(f'Training set target shape: {test_target.shape}')

Training set shape: (508401, 766)
Training set target shape: (508401,)
Test set shape: (217887, 766)
Training set target shape: (217887,)


In [10]:
test_target.value_counts()

False    217868
True         19
Name: TLJYWBE, dtype: int64

## Modeling

In [24]:
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold, RepeatedStratifiedKFold

In [15]:
dtypes = training_features.dtypes
dtypes.value_counts()

float64    741
int64       21
object       4
dtype: int64

In [16]:
int_features = dtypes[dtypes == 'int64'].index
float_features = dtypes[dtypes == 'float64'].index
cat_features = dtypes[dtypes == 'object'].index
len(int_features), len(float_features), len(cat_features)

(21, 741, 4)

In [17]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('int', numeric_transformer, int_features),
        ('float', numeric_transformer, float_features),
        ('cat', categorical_transformer, cat_features)
    ])

pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', 'passthrough')  
                          ])

## Balanced Random Forest

In [21]:
param_grid = [
    {
        'classifier': [BalancedRandomForestClassifier()],
        'classifier__depth': [None, 10, 20],
        'classifier__n_estimators': [100, 150, 200],
        'classifier__min_samples_leaf': [2, 4, 8],
    }
]

In [22]:
from src.custom_metrics import precision_at_full_recall

In [ ]:
%%time
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=1)
search = GridSearchCV(pipeline, param_grid, cv=cv, scoring=precision_at_full_recall, n_jobs=-1)
search.fit(training_features, train_target)

In [ ]:
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)